# Welcome

Authors:
- Célien Donzé, research assistant at Haute Ecole Arc Ingénierie, Switzerland
- Jonathan Guerne, research assistant at Haute Ecole Arc Ingénierie, Switzerland
- Henrique Marques Reis, research assistant at Haute Ecole Arc Ingénierie, Switzerland
- Pedro Costa, CO-Founder and CTO at Lumind, Switzerland

## Package installation

In [1]:
!pip install langchain langchain-community faiss-cpu pymupdf pypdf sentence_transformers rich wget

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.21.5-py3-none-manylinux2014_x86_64.whl.metadata (1

## Imports

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms.ollama import Ollama
import os
import json
from rich.console import Console
import zipfile
import os
import wget


console=Console()

# Downloading the pdfs

In [3]:
# Create the "data/PDFs" folder if it doesn't exist
os.makedirs("data/PDFs", exist_ok=True)

# Download the PDFs
url = "https://www.dropbox.com/scl/fo/xhqjzofiqnbmraxksgvlh/AAoL_WMBFOYDuipk5T_tTus?rlkey=qbbcvw4gbw6bpxkeijt6m94kt&st=yhap82wh&dl=1"
filename=wget.download(url, ".")

zip_file_path = f"./{filename}"
extract_folder = "data/PDFs"

# Create the extract folder if it doesn't exist
os.makedirs(extract_folder, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Pdf file downloaded successfully.")

Pdf file downloaded successfully.


## Documentation

- [langchain](https://python.langchain.com/v0.1/docs/get_started/introduction/)
- [Ollama website](https://ollama.com/)

## Constants

In [ ]:
OLLAMA_ADDRESS = "XXXXXX"
# OLLAMA_ADDRESS = "http://165.1.69.123:11434"
LLM_NAME="gemma"

# start step 1

## Connecting to LLM

In [5]:
llm= Ollama(
        model=LLM_NAME,
        base_url=OLLAMA_ADDRESS,
        temperature=0.1, # Will be explained later
        stop=["<end_of_turn>"],
    )

/tmp/ipykernel_938302/2651102297.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm= Ollama(


## Creating a prompt

A prompt is generally divided into two parts: the context and the question.

The context provides the information that the model will use to generate its answer, while the question specifies what the model is expected to respond to.

In a prompt, special characters are used to delineate different sections. For instance, in Gemma, these are `<start_of_turn>` and `<end_of_turn>`.

Additionally, LangChain requires markers indicating where to insert the user's question and the context retrieved from documents. For the question.

Gemma prompt template :

```html
<start_of_turn>user
{{ if .System }}{{ .System }} {{ end }}{{ .Prompt }}<end_of_turn>
<start_of_turn>model
{{ .Response }}<end_of_turn>
```

In [6]:
template = """<start_of_turn>
You are an helpful assistant that answer the question in detail.

Human input: {question}<end_of_turn>
<start_of_turn>Assistant:<end_of_turn>"""

prompt = PromptTemplate(input_variables=["question"], template=template)

## Creating the chain and start a conversation

In [7]:
conversation = LLMChain(
    llm=llm,
    # verbose=True, # uncomment if you want to see more information about the chain
    prompt=prompt
)

/tmp/ipykernel_938302/1867296296.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(


In [8]:
result = conversation.invoke(input="What is the capital of Switzerland?")
console.print(result.get("text"))

**The capital of Switzerland is Bern.**

**Background:**

* Bern is a city in western Switzerland.
* It has been the capital of Switzerland since 1848.
* The city is known for its historic old town, charming riverfront, and modern architecture.

**Key features of Bern as the capital:**

* **Government buildings:** The federal government of Switzerland is headquartered in Bern, including the Federal 
Council (parliament) and the Federal Chancellery (executive office).
* **Administrative center:** Bern is the central hub for government services and agencies.
* **Economic and cultural hub:** The city is a major economic and cultural center in Switzerland, with a strong 
financial sector and a vibrant arts and culture scene.
* **Historical significance:** Bern has a rich history dating back to the Middle Ages.

**Other notable facts about Bern:**

* It is the fourth-largest city in Switzerland by population.
* The city's official language is German.
* Bern is known for its chocolate production and has several chocolate factories.
* The Aare River flows through the city.

**Therefore, the capital of Switzerland is Bern, a historic and thriving city that serves as the political and 
economic heart of the country.**

# end step 1

# start step 2

## Loading a PDF

In [9]:
DATA_DIR = os.path.join("./", "data")
PDF_DIR = os.path.join(DATA_DIR, "PDFs")
VECTORSTORES_DIR = os.path.join(DATA_DIR, "vectorstores")

In [10]:
loader = PyPDFDirectoryLoader(PDF_DIR)
doc = loader.load()

## Embedding a PDF in a vectorstore

In [11]:
CHUNK_SIZE = 500
CHUNK_OVERLAP = 100
EMBEDDING_MODEL_NAME = "BAAI/bge-large-en-v1.5"

<div>
<img src="chunk_overlap_size_scheme.png" width="800"/>
</div>

In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
)

model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embedding_model = HuggingFaceBgeEmbeddings(
    model_name=EMBEDDING_MODEL_NAME, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [13]:
all_splits = text_splitter.split_documents(doc)
vectorstore = FAISS.from_documents(documents=all_splits,embedding=embedding_model)


In [14]:
vectorstore.save_local(VECTORSTORES_DIR)

# end step 2

# start step 3

## Loading a vectorstore

In [15]:
vectorstore = FAISS.load_local(VECTORSTORES_DIR,embedding_model,allow_dangerous_deserialization=True)

## What is temperature?

The temperature parameter in a language model (LLM) controls the randomness of the model's output.

A lower temperature value (closer to 0) makes the model more deterministic, favoring higher probability words and resulting in more predictable and repetitive text.

A higher temperature value (closer to 1) increases randomness, allowing for more creative and diverse responses by giving less probable words a better chance of being chosen.

Adjusting the temperature helps balance between coherence and creativity in the generated text.

## New prompt

In RAG we need to add another marker to indicate where the new information (or context) should be inserted for this we use the variable named `{context}`.

In [16]:
prompt= """<start_of_turn>
Use the following pieces of context to answer the question at the end.
Don't try to make up an answer and only use the information you know.
Use three sentences maximum and keep the answer as concise as possible.
You must answer in english.
{context}<end_of_turn>
<start_of_turn>{question}<end_of_turn>"""


prompt_template = PromptTemplate(input_variables=["context", "question"],template=prompt)

## Creating the chain

In [17]:
# Top k of chunks to retrieve from the vectorstore
NB_RETRIVED_CHUNKS = 8

In [18]:
rqa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(search_kwargs={"k": NB_RETRIVED_CHUNKS}),
        chain_type_kwargs={"prompt": prompt_template},
        input_key="question", # same as the variable in the prompt
        output_key="answer",
        return_source_documents=True,
    )

## Chatting with a pdf

In [19]:
result = rqa_chain.invoke("Can I fly to Graubunden and is it there a lot of sunshine ?")
result

{'question': 'Can I fly to Graubunden and is it there a lot of sunshine ?',
 'answer': 'Yes, you can fly to Graubunden. Zurich Airport is conveniently connected to the public transport network. The region enjoys plenty of sunshine, with Graubunden being one of the sunniest areas in Switzerland.',
 'source_documents': [Document(metadata={'source': 'data/PDFs/Quality_of_life_in_the_Grisons.pdf', 'page': 4}, page_content='24\nTips and Information \nCulture and leisure-time activities\nwww.graubuenden.ch\nWeather forecasts\nwww.wetter-graubuenden.ch \nPublic transport\nwww.rhb.ch | www.sbb.ch\n4. QUALITY OF LIFE AND LIFESTYLE\nHealth and Safety\nQuality of life: plenty of sunshine, no fog\nGood weather and a favourable climate improve wellbeing. \nThe Grisons is one of the areas with the most sunshine in \nSwitzerland and thanks to the long hours of sun and the \nmild climate, it is also the warmest vine growing area in the'),
  Document(metadata={'source': 'data/PDFs/2023-travel-brochure-

## Embellishing the output

In [20]:
def prepare_document(x):
        return x if x is None else os.path.basename(x)

def prepare_page(x):
        return x if x is None else int(x) + 1

def prepare_source(x):
        return {
            "document": prepare_document(x.metadata.get("source", None)),
            "page": prepare_page(x.metadata.get("page", None)),
            "chunk": x.page_content,
        }

In [21]:
console.print(result.get("answer"))

Yes, you can fly to Graubunden. Zurich Airport is conveniently connected to the public transport network. The 
region enjoys plenty of sunshine, with Graubunden being one of the sunniest areas in Switzerland.

In [22]:
sources = [prepare_source(x) for x in result["source_documents"]]
console.print(json.dumps(sources, indent=1),highlight=False)

[
 {
  "document": "Quality_of_life_in_the_Grisons.pdf",
  "page": 5,
  "chunk": "24\nTips and Information \nCulture and leisure-time activities\nwww.graubuenden.ch\nWeather 
forecasts\nwww.wetter-graubuenden.ch \nPublic transport\nwww.rhb.ch | www.sbb.ch\n4. QUALITY OF LIFE AND 
LIFESTYLE\nHealth and Safety\nQuality of life: plenty of sunshine, no fog\nGood weather and a favourable climate 
improve wellbeing. \nThe Grisons is one of the areas with the most sunshine in \nSwitzerland and thanks to the long
hours of sun and the \nmild climate, it is also the warmest vine growing area in the"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 5,
  "chunk": "How to get there\nFlying to Graubunden\nZurich Airport is one of Europe\u2019s air traffic  hubs and is
served by numerous \nairlines. Additionally, it is ideally connected to the  public transport network, 
\nguaranteeing a  seamless post-flight journey to  Graubunden.\nNumerous airports in and around  Graubunden include
Friedrichshafen (DE),  \nInnsbruck (AUT), St. Gallen-Altenrhein,  Basel, and Samedan near St. Moritz.\n \nDriving 
to Graubunden"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 1,
  "chunk": "THE NO.1 HOLIDAY DESTINATION  \nIN SWITZERLAND\nVisit \nGraubunden\nTravel Brochure\nPhoto by\n\u00a9 
Flims Laax Falera / Gaudenz Danuser"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 2,
  "chunk": "Welcome to Graubunden\nPerfect for families seeking spectacular outdoor\nadventure, each region of the 
canton is only a\nhop, skip, and a jump away. \nTaking the title for the easternmost and largest\ncanton of 
Switzerland - bordering Austria, Italy,\nand Liechtenstein - when asked what makes this\nplace so special, locals 
wax lyrical about its wild,\nnatural beauty coupled with the finer things in life.\nA place of bright green 
valleys, snow-capped\npeaks and shimmering Alpine lakes, days are"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 2,
  "chunk": "been titled among the most spectacular train\njourneys in the world.\nPhoto by\n\u00a9 \u00a9 Daniel 
Geiger\nEven with Switzerland\u2019s competitive repertoire\nof awe-inspiring scenery, delicious cheese 
and\nchocolate, world-class medical spas, and leading\nboarding schools, the canton (state) of Graubunden \nstands 
out. Renowned as one of the grandest of the \nSwiss Alpine Valleys, its supermodel looks are home \nto the classic 
children\u2019s novel, Heidi, the sparkly ski"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 2,
  "chunk": "to the classic children\u2019s novel, Heidi, the sparkly ski \nresorts of St. Moritz, and the 
shimmering Lake  \nCaumasee - one of Switzerland\u2019s most beautiful lakes. \nHighest mountain \nPiz Bernina \nat
13,284 ft\nWelcome to Graubunden  3\nMap of Graubunden  6\nHow to get there  9\nActivities & experiences  
10\nDestinations and hotels  28\nImpressum/Contact  72\nWhat's  \ninside?\n3\n 3\n 6\n 9\n 10\n 28\n \n 72"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 5,
  "chunk": "Driving to Graubunden\nIf you are coming from another European  destination, reaching Graubunden \nby 
road  is an excellent way to get into vacation  mode. Graubunden\u2019s vast \nnetwork of well-maintained roads and
highways make  traveling around easy.\nOur route planner shows you the fastest  way to reach your vacation  
\ndestination. Graubunden has reliable roadside  assistance with major brands \nplus  reliable cell phone 
coverage.\nTraveling by train"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 6,
  "chunk": "Photo by\n\u00a9 Graubunden Tourism\nArosa Barenland T oboggan Run Globi Hiking Trail\nFamily\nSafe, 
beautiful and easy to  \ntraverse, Graubunden is a dream \nfor family travel. Kids will revel in \nrushing down 
Switzerland\u2019s  \nlongest toboggan run, rel